In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
import pandas_ta as ta  # ATR 계산용 (없으면 pip install pandas_ta)

# ============================================================
# [설정] 현물(Spot) 백테스트 환경
# ============================================================
BASE_DIR = Path("2025-11-27_Sniper")
# ★ 중요: 원본 4시간봉 데이터 경로 (파일명 확인 필수)
ORIGINAL_DATA_PATH = Path("../eth_4hour.csv") 

FEE_RATE = 0.0005       # 0.05% (업비트 기준)
INITIAL_EQUITY = 1.0    # 시작 자산 1.0 (100%)
MODELS = ["CatBoost", "XGBoost", "LightGBM", "RandomForest"]

# [현물 자금 관리 설정]
MAX_PYRAMIDING = 2      # 자산을 2등분 (포지션당 50% 투입) - 공격적 세팅

# ============================================================
# [함수 1] 지표 계산 헬퍼 (승률 추가)
# ============================================================
def calculate_metrics(equity_curve, trades_pnl):
    """
    equity_curve: 자산 곡선 (Series)
    trades_pnl: 개별 트레이드의 수익률 리스트 (승률 계산용)
    """
    if len(equity_curve) < 2:
        return {"Return": 0.0, "MDD": 0.0, "Sharpe": 0.0, "WinRate": 0.0, "Trades": 0}

    # 1. 수익률
    final_return = equity_curve.iloc[-1] / equity_curve.iloc[0] - 1.0
    
    # 2. MDD
    roll_max = equity_curve.cummax()
    drawdown = equity_curve / roll_max - 1.0
    mdd = drawdown.min()
    
    # 3. Sharpe
    pct_change = equity_curve.pct_change().dropna()
    if len(pct_change) > 1 and pct_change.std() > 0:
        sharpe = (pct_change.mean() / pct_change.std()) * np.sqrt(len(pct_change))
    else:
        sharpe = 0.0
        
    # 4. Win Rate & Trades
    if len(trades_pnl) > 0:
        wins = sum(1 for pnl in trades_pnl if pnl > 0)
        win_rate = wins / len(trades_pnl)
    else:
        win_rate = 0.0
        
    return {
        "Return": final_return, 
        "MDD": mdd, 
        "Sharpe": sharpe, 
        "WinRate": win_rate, 
        "Trades": len(trades_pnl)
    }

def calculate_bh_metrics_fold_cumulative(df, fee_rate=0.0005):
    """
    각 Fold별 B&H 수익률을 복리로 연결 (벤치마크용)
    """
    if "close" not in df.columns:
        return {"Return": 0.0, "MDD": 0.0, "Sharpe": 0.0}

    bh_equity = [1.0]
    current_eq = 1.0
    df = df.sort_values("timestamp")
    
    for fold_name, group in df.groupby("fold"):
        group = group.sort_values("timestamp")
        if len(group) < 2: continue
        
        start_p = group.iloc[0]["close"]
        end_p = group.iloc[-1]["close"]
        
        # B&H는 매수 1회, 매도 1회 수수료 반영
        raw_ret = (end_p - start_p) / start_p
        net_ret = (1 + raw_ret) * (1 - fee_rate)**2 - 1
        
        current_eq *= (1 + net_ret)
        bh_equity.append(current_eq)
        
    # B&H는 트레이드 횟수 개념이 모호하므로 Return/MDD 위주로 계산
    return calculate_metrics(pd.Series(bh_equity), [])


# ============================================================
# [함수 2] 데이터 복원 (Predictions + Original Price)
# ============================================================
def prepare_original_data(csv_path):
    if not csv_path.exists():
        potentials = list(Path(".").rglob(csv_path.name))
        if potentials:
            csv_path = potentials[0]
        else:
            raise FileNotFoundError(f"원본 데이터가 없습니다: {csv_path}")
        
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.lower()
    
    date_col = 'timestamp' if 'timestamp' in df.columns else 'date'
    df[date_col] = pd.to_datetime(df[date_col])
    
    h = df.get('high', df.get('eth_high'))
    l = df.get('low', df.get('eth_low'))
    c = df.get('close', df.get('eth_close'))
    o = df.get('open', df.get('eth_open'))
    
    df['atr_84'] = ta.atr(h, l, c, length=84)
    df['next_open'] = o.shift(-1)
    df['orig_close'] = c
    
    return df[[date_col, 'atr_84', 'next_open', 'orig_close']].rename(columns={date_col: 'timestamp'})

def restore_and_merge(preds_df, orig_data_df, profit_mult, stop_mult):
    df = preds_df.copy()
    if "timestamp" not in df.columns: return df
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    
    df = pd.merge(df, orig_data_df, on="timestamp", how="left")
    
    # 가격 복원
    fallback_close = df.get("orig_close", df.get("close"))
    if "real_entry_price" not in df.columns:
        df["real_entry_price"] = df["next_open"].fillna(fallback_close)
    else:
        df["real_entry_price"] = df["real_entry_price"].fillna(df["next_open"]).fillna(fallback_close)

    # TP/SL 복원
    if "take_profit_price" not in df.columns:
        atr = df["atr_84"].fillna(0)
        entry = df["real_entry_price"]
        df["take_profit_price"] = entry + (atr * profit_mult)
        df["stop_loss_price"] = entry - (atr * stop_mult)
        
    return df


# ============================================================
# [함수 3] 현물(Spot) 전용 백테스트 로직
# ============================================================
def backtest_spot_pyramiding(df, fee_rate, max_pos):
    df = df.sort_values("timestamp").reset_index(drop=True)
    
    cash = INITIAL_EQUITY
    positions = [] 
    equity_curve = [INITIAL_EQUITY]
    trades_pnl = [] # 승률 계산용
    
    for i, row in df.iterrows():
        curr_price = row["real_entry_price"]
        if pd.isna(curr_price) or curr_price <= 0:
            equity_curve.append(equity_curve[-1])
            continue

        # 1. 포지션 관리
        active_pos = []
        for pos in positions:
            # 익절
            if curr_price >= pos['tp']:
                # 시가 체결 가정 (보수적으로 TP가격 체결이 아닌, 시가 체결로 수익 확정)
                # 실제로는 TP가격에 지정가 매도를 걸어두므로 max(curr, tp)가 맞음
                sell_price = max(curr_price, pos['tp'])
                
                sell_val = pos['amount'] * sell_price
                fee = sell_val * fee_rate
                net_cash = sell_val - fee
                
                # 수익률 기록 (단리)
                entry_val = pos['amount'] * pos['entry']
                pnl_pct = (net_cash - entry_val) / entry_val
                trades_pnl.append(pnl_pct)
                
                cash += net_cash
                continue
                
            # 손절
            if curr_price <= pos['sl']:
                sell_price = min(curr_price, pos['sl'])
                
                sell_val = pos['amount'] * sell_price
                fee = sell_val * fee_rate
                net_cash = sell_val - fee
                
                entry_val = pos['amount'] * pos['entry']
                pnl_pct = (net_cash - entry_val) / entry_val
                trades_pnl.append(pnl_pct)
                
                cash += net_cash
                continue
                
            active_pos.append(pos)
        positions = active_pos
        
        # 2. 신규 진입
        sig = int(row.get("signal", 0))
        if sig == 1 and len(positions) < max_pos:
            entry = float(row["real_entry_price"])
            sl = float(row["stop_loss_price"])
            tp = float(row["take_profit_price"])
            
            if entry > 0:
                # 현재 총 자산
                curr_equity = cash + sum([p['amount']*curr_price for p in positions])
                target_invest = curr_equity / max_pos
                
                if cash >= target_invest * 0.99:
                    buy_amt = target_invest
                    fee = buy_amt * fee_rate
                    real_buy_amt = buy_amt - fee
                    amount = real_buy_amt / entry
                    
                    cash -= buy_amt
                    positions.append({'entry': entry, 'tp': tp, 'sl': sl, 'amount': amount})

        # 3. 자산 평가
        coin_val = sum([p['amount'] * curr_price for p in positions])
        total_equity = cash + coin_val
        equity_curve.append(total_equity)

    return calculate_metrics(pd.Series(equity_curve), trades_pnl)


# ============================================================
# [Main]
# ============================================================
def load_preds(trial_dir, model_name):
    dfs = []
    for sub in trial_dir.iterdir():
        if not sub.is_dir(): continue
        meta_path = sub / "metadata.json"
        if not meta_path.exists(): continue
        with open(meta_path) as f: meta = json.load(f)
        
        p_path = sub / f"predictions_{model_name}.csv"
        if not p_path.exists(): continue
        
        df = pd.read_csv(p_path)
        if "timestamp" not in df.columns: df = df.reset_index().rename(columns={"index":"timestamp"})
        df["fold"] = sub.name
        df["fold_type"] = meta.get("fold_type", "unknown")
        dfs.append(df)
    
    if not dfs: return None
    return pd.concat(dfs).sort_values("timestamp").reset_index(drop=True)

if __name__ == "__main__":
    print("1. Loading Original Data...")
    try:
        orig_data = prepare_original_data(ORIGINAL_DATA_PATH)
        print("   -> Success!")
    except Exception as e:
        print(f"   -> Error: {e}")
        exit()

    dev_results = []
    hold_results = []
    
    print(f"\n2. Starting SPOT Backtest (Max {MAX_PYRAMIDING} Pos)...")

    for trial in sorted(BASE_DIR.iterdir()):
        if not trial.is_dir() or not trial.name.startswith("T"): continue
        try:
            with open(next(trial.glob("*/metadata.json"))) as f:
                params = json.load(f).get("trial_params", {})
            p_mult = float(params.get("profit_mult", 1.6))
            s_mult = float(params.get("stop_mult", 0.5))
        except: continue

        print(f"   Processing {trial.name}...")

        for model in MODELS:
            df = load_preds(trial, model)
            if df is None: continue
            df = restore_and_merge(df, orig_data, p_mult, s_mult)
            
            dev_df = df[df["fold_type"] != "final_holdout"]
            hold_df = df[df["fold_type"] == "final_holdout"]
            
            # 백테스트
            d_res = backtest_spot_pyramiding(dev_df, FEE_RATE, MAX_PYRAMIDING)
            d_bh = calculate_bh_metrics_fold_cumulative(dev_df, FEE_RATE)
            
            dev_results.append({
                "Trial": trial.name, "Model": model,
                "Strat_Ret": d_res["Return"], "BH_Ret": d_bh["Return"],
                "Strat_MDD": d_res["MDD"], "Strat_Win": d_res["WinRate"], "Strat_Cnt": d_res["Trades"]
            })
            
            if len(hold_df) > 0:
                h_res = backtest_spot_pyramiding(hold_df, FEE_RATE, MAX_PYRAMIDING)
                h_bh = calculate_bh_metrics_fold_cumulative(hold_df, FEE_RATE)
                
                hold_results.append({
                    "Trial": trial.name, "Model": model,
                    "Strat_Ret": h_res["Return"], "BH_Ret": h_bh["Return"],
                    "Strat_MDD": h_res["MDD"], "Strat_Win": h_res["WinRate"], "Strat_Cnt": h_res["Trades"]
                })

    # ============================================================
    # [출력]
    # ============================================================
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.width', 200) # 넓게 보기
    
    def format_df(df):
        out = df.copy()
        # 퍼센트 포맷팅
        for c in ["Strat_Ret", "BH_Ret", "Strat_MDD", "Strat_Win"]:
            if c in out.columns:
                out[c] = out[c].apply(lambda x: f"{x*100:.2f}%")
        return out

    print("\n" + "="*120)
    print("[ANALYSIS 1] Walk-Forward (Dev) - SPOT Strategy")
    print("="*120)
    if dev_results:
        df_dev = pd.DataFrame(dev_results).sort_values("Strat_Ret", ascending=False)
        print(format_df(df_dev).head(10).to_string(index=False))
        df_dev.to_csv(BASE_DIR / "summary_spot_dev.csv", index=False)

    print("\n" + "="*120)
    print("[ANALYSIS 2] Final Holdout - SPOT Strategy")
    print("="*120)
    if hold_results:
        df_hold = pd.DataFrame(hold_results).sort_values("Strat_Ret", ascending=False)
        print(format_df(df_hold).head(10).to_string(index=False))
        df_hold.to_csv(BASE_DIR / "summary_spot_holdout.csv", index=False)

    print(f"\nFiles saved to {BASE_DIR}")


1. Loading Original Data...
   -> Success!

2. Starting SPOT Backtest (Max 2 Pos)...
   Processing T0_L12_P1.0_S0.5_N30...
   Processing T0_L3_P1.2_S0.6_N20...
   Processing T0_L9_P1.4_S0.9_N30...
   Processing T10_L9_P1.8_S1.0_N30...
   Processing T12_L9_P1.8_S0.6_N30...
   Processing T13_L6_P1.8_S1.0_N25...
   Processing T14_L9_P1.0_S0.7_N25...
   Processing T15_L12_P1.8_S0.5_N25...
   Processing T16_L12_P1.4_S0.6_N25...
   Processing T17_L12_P1.8_S0.7_N30...
   Processing T18_L6_P1.4_S0.9_N20...
   Processing T19_L12_P1.8_S0.5_N15...
   Processing T1_L3_P1.2_S0.5_N10...
   Processing T1_L9_P1.2_S0.7_N15...
   Processing T22_L9_P1.8_S0.5_N20...
   Processing T23_L6_P1.6_S0.6_N25...
   Processing T24_L9_P1.8_S0.5_N30...
   Processing T25_L9_P1.6_S0.6_N30...
   Processing T26_L12_P1.8_S0.5_N30...
   Processing T27_L12_P1.0_S0.7_N30...
   Processing T28_L12_P1.6_S0.5_N30...
   Processing T29_L12_P1.6_S0.9_N30...
   Processing T2_L3_P1.6_S0.8_N10...
   Processing T30_L12_P1.6_S0.6_N30...

In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
import pandas_ta as ta
from itertools import combinations

# ============================================================
# [설정]
# ============================================================
BASE_DIR = Path("2025-11-27_Sniper")
ORIGINAL_DATA_PATH = Path("../eth_4hour.csv") 

FEE_RATE = 0.002
INITIAL_EQUITY = 1.0
MAX_PYRAMIDING = 2 
MODELS = ["CatBoost", "XGBoost", "LightGBM", "RandomForest"] # A, B, C, D

# ============================================================
# [함수]
# ============================================================
def prepare_original_data(csv_path):
    if not csv_path.exists():
        potentials = list(Path(".").rglob(csv_path.name))
        if potentials: csv_path = potentials[0]
        else: raise FileNotFoundError(f"Missing: {csv_path}")
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.lower()
    date_col = 'timestamp' if 'timestamp' in df.columns else 'date'
    df[date_col] = pd.to_datetime(df[date_col])
    h, l, c, o = df.get('high',df.get('eth_high')), df.get('low',df.get('eth_low')), df.get('close',df.get('eth_close')), df.get('open',df.get('eth_open'))
    df['atr_84'] = ta.atr(h, l, c, length=84)
    df['next_open'] = o.shift(-1)
    df['orig_close'] = c
    return df[[date_col, 'atr_84', 'next_open', 'orig_close']].rename(columns={date_col: 'timestamp'})

def restore_and_merge(preds, orig, p_mult, s_mult):
    df = preds.copy()
    if "timestamp" not in df.columns: return df
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = pd.merge(df, orig, on="timestamp", how="left")
    fb = df.get("orig_close", df.get("close"))
    if "real_entry_price" not in df.columns: df["real_entry_price"] = df["next_open"].fillna(fb)
    else: df["real_entry_price"] = df["real_entry_price"].fillna(df["next_open"]).fillna(fb)
    if "take_profit_price" not in df.columns:
        atr = df["atr_84"].fillna(0)
        entry = df["real_entry_price"]
        df["take_profit_price"] = entry + (atr * p_mult)
        df["stop_loss_price"] = entry - (atr * s_mult)
    return df

def calculate_metrics(equity_curve, trades_pnl=[]):
    if len(equity_curve) < 2: return {"Return": 0.0, "MDD": 0.0}
    ret = equity_curve.iloc[-1] / equity_curve.iloc[0] - 1.0
    mdd = (equity_curve / equity_curve.cummax() - 1.0).min()
    return {"Return": ret, "MDD": mdd}

def calculate_bh_metrics(df, fee_rate=0.0005):
    if "close" not in df.columns: return {"Return": 0.0}
    bh = [1.0]
    curr = 1.0
    df = df.sort_values("timestamp")
    for f, g in df.groupby("fold"):
        g = g.sort_values("timestamp")
        if len(g)<2: continue
        s, e = g.iloc[0]["close"], g.iloc[-1]["close"]
        curr *= (1 + ((e-s)/s)) * (1-fee_rate)**2
        bh.append(curr)
    return calculate_metrics(pd.Series(bh))

def backtest_spot_pyramiding(df, fee_rate, max_pos, signal_col="signal"):
    df = df.sort_values("timestamp").reset_index(drop=True)
    cash = INITIAL_EQUITY
    positions = []
    equity_curve = [INITIAL_EQUITY]
    
    for i, row in df.iterrows():
        curr = row["real_entry_price"]
        if pd.isna(curr) or curr <= 0:
            equity_curve.append(equity_curve[-1])
            continue
        
        active = []
        for p in positions:
            if curr >= p['tp']:
                sell_val = p['amt'] * max(curr, p['tp'])
                cash += sell_val * (1 - fee_rate)
                continue
            if curr <= p['sl']:
                sell_val = p['amt'] * min(curr, p['sl'])
                cash += sell_val * (1 - fee_rate)
                continue
            active.append(p)
        positions = active
        
        sig = int(row.get(signal_col, 0))
        if sig == 1 and len(positions) < max_pos:
            entry, sl, tp = row["real_entry_price"], row["stop_loss_price"], row["take_profit_price"]
            if entry > 0:
                curr_eq = cash + sum([p['amt']*curr for p in positions])
                invest = curr_eq / max_pos
                if cash >= invest * 0.99:
                    buy_amt = invest * (1 - fee_rate)
                    positions.append({'entry': entry, 'tp': tp, 'sl': sl, 'amt': buy_amt/entry})
                    cash -= invest
        
        val = cash + sum([p['amt']*curr for p in positions])
        equity_curve.append(val)
        
    return calculate_metrics(pd.Series(equity_curve))

def load_single_prediction(trial_dir, model):
    dfs = []
    for sub in trial_dir.iterdir():
        if not sub.is_dir(): continue
        p_path = sub / f"predictions_{model}.csv"
        if p_path.exists():
            d = pd.read_csv(p_path)
            if "timestamp" not in d.columns: d = d.reset_index().rename(columns={"index":"timestamp"})
            d["fold"] = sub.name
            with open(sub/"metadata.json") as f: d["fold_type"] = json.load(f).get("fold_type", "unknown")
            dfs.append(d)
    if not dfs: return None
    return pd.concat(dfs).sort_values("timestamp").reset_index(drop=True)


# ============================================================
# [Main Logic]
# ============================================================
if __name__ == "__main__":
    print("1. Loading Original Data...")
    orig_data = prepare_original_data(ORIGINAL_DATA_PATH)
    
    all_competitors = []
    
    print("\n2. Starting Full League (All Combinations)...")
    
    for trial in sorted(BASE_DIR.iterdir()):
        if not trial.is_dir() or not trial.name.startswith("T"): continue
        try:
            with open(next(trial.glob("*/metadata.json"))) as f: p = json.load(f).get("trial_params", {})
            pm, sm = float(p.get("profit_mult", 1.6)), float(p.get("stop_mult", 0.5))
        except: continue
        
        print(f"   Scanning {trial.name}...")
        
        # 1. 데이터 로드
        trial_models_data = {}
        valid = True
        for model in MODELS:
            df = load_single_prediction(trial, model)
            if df is None: valid = False; break
            df = restore_and_merge(df, orig_data, pm, sm)
            trial_models_data[model] = df
        if not valid: continue

        # 베이스 데이터프레임 (시그널 통합용)
        base_df = trial_models_data[MODELS[0]].copy()
        for model in MODELS:
            sig = trial_models_data[model][["timestamp", "signal"]].rename(columns={"signal": f"sig_{model}"})
            base_df = pd.merge(base_df, sig, on="timestamp", how="left").fillna(0)

        # Dev / Holdout 분리
        dev_base = base_df[base_df["fold_type"] != "final_holdout"]
        hold_base = base_df[base_df["fold_type"] == "final_holdout"]
        if len(dev_base) == 0: continue

        # B&H (공통)
        dev_bh = calculate_bh_metrics(dev_base, FEE_RATE)
        hold_bh = calculate_bh_metrics(hold_base, FEE_RATE)

        # ============================================================
        # [조합 생성 및 백테스트 Loop]
        # ============================================================
        # 1. 단일 모델 (Size 1)
        # 2. 조합 모델 (Size 2, 3, 4)
        
        # 모든 가능한 조합 리스트 생성 (1개짜리 포함)
        all_combos = []
        for r in range(1, len(MODELS) + 1):
            all_combos.extend(combinations(MODELS, r))
            
        for combo in all_combos:
            combo_name = "+".join(combo)
            col_name = f"ens_{combo_name}"
            
            # 시그널 생성
            # 1개짜리면 그냥 그 모델 시그널
            # 2개 이상이면: 과반수 투표 (2개면 2, 3개면 2, 4개면 3)
            # Threshold: (N // 2) + 1
            n = len(combo)
            threshold = (n // 2) + 1
            
            vote_sum = base_df[[f"sig_{m}" for m in combo]].sum(axis=1)
            base_df[col_name] = (vote_sum >= threshold).astype(int)
            
            # 백테스트 (Dev)
            dev_sub = base_df[base_df["fold_type"] != "final_holdout"]
            res = backtest_spot_pyramiding(dev_sub, FEE_RATE, MAX_PYRAMIDING, signal_col=col_name)
            
            # 후보 등록
            all_competitors.append({
                "Name": f"{trial.name} | {combo_name} ({n})",
                "Combo": combo,
                "Trial": trial.name,
                "Signal_Col": col_name,
                "Dev_Ret": res["Return"], "Dev_MDD": res["MDD"],
                "Dev_BH": dev_bh["Return"],
                "Hold_BH": hold_bh["Return"],
                "Base_Data": base_df # 전체 데이터 저장 (Holdout용)
            })

    # ============================================================
    # [순위 선정] Top 20 (Dev 수익률 기준)
    # ============================================================
    print("\n3. Selecting Top 20 Models (Based on Dev Return)...")
    top20 = sorted(all_competitors, key=lambda x: x["Dev_Ret"], reverse=True)[:20]
    
    # ============================================================
    # [Final Holdout 검증]
    # ============================================================
    print("4. Running Final Holdout Test for Top 20...")
    final_results = []
    
    for p in top20:
        df = p["Base_Data"]
        hold = df[df["fold_type"] == "final_holdout"]
        
        h_res = {"Return": 0.0, "MDD": 0.0}
        if len(hold) > 0:
            h_res = backtest_spot_pyramiding(hold, FEE_RATE, MAX_PYRAMIDING, signal_col=p["Signal_Col"])
            
        final_results.append({
            "Model Name": p["Name"],
            "Dev Return": p["Dev_Ret"],
            "Dev MDD": p["Dev_MDD"],
            "Holdout Return": h_res["Return"],
            "Holdout MDD": h_res["MDD"],
            "Holdout B&H": p["Hold_BH"]
        })
        
    # ============================================================
    # [최종 출력]
    # ============================================================
    res_df = pd.DataFrame(final_results)
    print_df = res_df.copy()
    for c in print_df.columns:
        if "Return" in c or "MDD" in c or "B&H" in c:
            print_df[c] = print_df[c].apply(lambda x: f"{x*100:.2f}%" if isinstance(x, (int, float)) else x)
            
    print("\n" + "="*120)
    print("🏆 FINAL LEAGUE RESULTS: All Combinations")
    print("="*120)
    print(print_df.head(20).to_string(index=False))
    
    res_df.to_csv(BASE_DIR / "sniper_full_ensemble_results.csv", index=False)


1. Loading Original Data...

2. Starting Full League (All Combinations)...
   Scanning T0_L12_P1.0_S0.5_N30...
   Scanning T0_L3_P1.2_S0.6_N20...
   Scanning T0_L9_P1.4_S0.9_N30...
   Scanning T10_L9_P1.8_S1.0_N30...
   Scanning T12_L9_P1.8_S0.6_N30...
   Scanning T13_L6_P1.8_S1.0_N25...
   Scanning T14_L9_P1.0_S0.7_N25...
   Scanning T15_L12_P1.8_S0.5_N25...
   Scanning T16_L12_P1.4_S0.6_N25...
   Scanning T17_L12_P1.8_S0.7_N30...
   Scanning T18_L6_P1.4_S0.9_N20...
   Scanning T19_L12_P1.8_S0.5_N15...
   Scanning T1_L3_P1.2_S0.5_N10...
   Scanning T1_L9_P1.2_S0.7_N15...
   Scanning T22_L9_P1.8_S0.5_N20...
   Scanning T23_L6_P1.6_S0.6_N25...
   Scanning T24_L9_P1.8_S0.5_N30...
   Scanning T25_L9_P1.6_S0.6_N30...
   Scanning T26_L12_P1.8_S0.5_N30...
   Scanning T27_L12_P1.0_S0.7_N30...
   Scanning T28_L12_P1.6_S0.5_N30...
   Scanning T29_L12_P1.6_S0.9_N30...
   Scanning T2_L3_P1.6_S0.8_N10...
   Scanning T30_L12_P1.6_S0.6_N30...
   Scanning T31_L12_P1.8_S0.5_N30...
   Scanning T34_L12_

In [7]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
import pandas_ta as ta
from itertools import combinations
import os

# ============================================================
# [설정]
# ============================================================
BASE_DIR = Path("2025-11-27_Sniper")
REPORT_DIR = BASE_DIR / "analysis_reports" # 리포트 저장 폴더
ORIGINAL_DATA_PATH = Path("../eth_4hour.csv") 

FEE_RATE = 0.002 # 0.2% (보수적)
INITIAL_EQUITY = 1.0
MAX_PYRAMIDING = 2 
MODELS = ["CatBoost", "XGBoost", "LightGBM", "RandomForest"]

# ============================================================
# [함수] 지표 계산 (업그레이드)
# ============================================================
def calculate_metrics_detailed(equity_curve, trades_pnl=[]):
    if len(equity_curve) < 2: 
        return {"Return": 0.0, "MDD": 0.0, "Sharpe": 0.0, "WinRate": 0.0, "Trades": 0}
    
    ret = equity_curve.iloc[-1] / equity_curve.iloc[0] - 1.0
    mdd = (equity_curve / equity_curve.cummax() - 1.0).min()
    
    pct = equity_curve.pct_change().dropna()
    # Sharpe (연환산 아님, 단순 비율)
    sharpe = (pct.mean() / pct.std()) * np.sqrt(len(pct)) if len(pct) > 1 and pct.std() > 0 else 0.0
    
    win_rate = sum(1 for p in trades_pnl if p > 0) / len(trades_pnl) if trades_pnl else 0.0
    
    return {"Return": ret, "MDD": mdd, "Sharpe": sharpe, "WinRate": win_rate, "Trades": len(trades_pnl)}

def calculate_bh_metrics_detailed(df, fee_rate):
    if "close" not in df.columns or len(df) < 2:
        return {"Return": 0.0, "MDD": 0.0, "Sharpe": 0.0}
    
    s, e = df.iloc[0]["close"], df.iloc[-1]["close"]
    # B&H 수익률 (수수료 2회 반영)
    ret = ((e - s) / s + 1) * (1 - fee_rate)**2 - 1.0
    
    # MDD & Sharpe (가격 데이터로 직접 계산)
    prices = df["close"]
    mdd = (prices / prices.cummax() - 1.0).min()
    pct = prices.pct_change().dropna()
    sharpe = (pct.mean() / pct.std()) * np.sqrt(len(pct)) if len(pct) > 1 and pct.std() > 0 else 0.0
    
    return {"Return": ret, "MDD": mdd, "Sharpe": sharpe}

# ... (데이터 준비/복원 함수들은 이전과 동일 - 생략 가능하지만 실행용으로 포함) ...
def prepare_original_data(csv_path):
    if not csv_path.exists():
        potentials = list(Path(".").rglob(csv_path.name))
        if potentials: csv_path = potentials[0]
        else: raise FileNotFoundError(f"Missing: {csv_path}")
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.lower()
    date_col = 'timestamp' if 'timestamp' in df.columns else 'date'
    df[date_col] = pd.to_datetime(df[date_col])
    h, l, c, o = df.get('high',df.get('eth_high')), df.get('low',df.get('eth_low')), df.get('close',df.get('eth_close')), df.get('open',df.get('eth_open'))
    df['atr_84'] = ta.atr(h, l, c, length=84)
    df['next_open'] = o.shift(-1)
    df['orig_close'] = c
    return df[[date_col, 'atr_84', 'next_open', 'orig_close']].rename(columns={date_col: 'timestamp'})

def restore_and_merge(preds, orig, p_mult, s_mult):
    df = preds.copy()
    if "timestamp" not in df.columns: return df
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = pd.merge(df, orig, on="timestamp", how="left")
    fb = df.get("orig_close", df.get("close"))
    if "real_entry_price" not in df.columns: df["real_entry_price"] = df["next_open"].fillna(fb)
    else: df["real_entry_price"] = df["real_entry_price"].fillna(df["next_open"]).fillna(fb)
    if "take_profit_price" not in df.columns:
        atr = df["atr_84"].fillna(0)
        entry = df["real_entry_price"]
        df["take_profit_price"] = entry + (atr * p_mult)
        df["stop_loss_price"] = entry - (atr * s_mult)
    return df

def backtest_spot_pyramiding(df, fee_rate, max_pos, signal_col="signal", return_pnl_list=False):
    df = df.sort_values("timestamp").reset_index(drop=True)
    cash = INITIAL_EQUITY
    positions = []
    equity_curve = [INITIAL_EQUITY]
    trades_pnl = []
    
    for i, row in df.iterrows():
        curr = row["real_entry_price"]
        if pd.isna(curr) or curr <= 0:
            equity_curve.append(equity_curve[-1])
            continue
        
        active = []
        for p in positions:
            pnl = 0
            sold = False
            if curr >= p['tp']:
                sell_val = p['amt'] * max(curr, p['tp'])
                cash += sell_val * (1 - fee_rate)
                pnl = (sell_val*(1-fee_rate) - p['amt']*p['entry']) / (p['amt']*p['entry'])
                sold = True
            elif curr <= p['sl']:
                sell_val = p['amt'] * min(curr, p['sl'])
                cash += sell_val * (1 - fee_rate)
                pnl = (sell_val*(1-fee_rate) - p['amt']*p['entry']) / (p['amt']*p['entry'])
                sold = True
            
            if sold: trades_pnl.append(pnl)
            else: active.append(p)
        positions = active
        
        sig = int(row.get(signal_col, 0))
        if sig == 1 and len(positions) < max_pos:
            entry, sl, tp = row["real_entry_price"], row["stop_loss_price"], row["take_profit_price"]
            if entry > 0:
                curr_eq = cash + sum([p['amt']*curr for p in positions])
                invest = curr_eq / max_pos
                if cash >= invest * 0.99:
                    buy_amt = invest * (1 - fee_rate)
                    positions.append({'entry': entry, 'tp': tp, 'sl': sl, 'amt': buy_amt/entry})
                    cash -= invest
        
        val = cash + sum([p['amt']*curr for p in positions])
        equity_curve.append(val)
    
    if return_pnl_list: return trades_pnl
    return calculate_metrics(pd.Series(equity_curve), trades_pnl)

def load_single_prediction(trial_dir, model):
    dfs = []
    for sub in trial_dir.iterdir():
        if not sub.is_dir(): continue
        p_path = sub / f"predictions_{model}.csv"
        if p_path.exists():
            d = pd.read_csv(p_path)
            if "timestamp" not in d.columns: d = d.reset_index().rename(columns={"index":"timestamp"})
            d["fold"] = sub.name
            with open(sub/"metadata.json") as f: d["fold_type"] = json.load(f).get("fold_type", "unknown")
            dfs.append(d)
    if not dfs: return None
    return pd.concat(dfs).sort_values("timestamp").reset_index(drop=True)


# ============================================================
# [Main Logic]
# ============================================================
if __name__ == "__main__":
    if not REPORT_DIR.exists(): os.makedirs(REPORT_DIR)
    
    print("1. Loading Original Data...")
    orig_data = prepare_original_data(ORIGINAL_DATA_PATH)
    
    # 1. 전체 경쟁자 생성 (단일 + 앙상블)
    all_competitors = []
    print("\n2. Creating Competitors...")
    
    for trial in sorted(BASE_DIR.iterdir()):
        if not trial.is_dir() or not trial.name.startswith("T"): continue
        try:
            with open(next(trial.glob("*/metadata.json"))) as f: p = json.load(f).get("trial_params", {})
            pm, sm = float(p.get("profit_mult", 1.6)), float(p.get("stop_mult", 0.5))
        except: continue
        
        print(f"   Processing {trial.name}...", end="\r")
        
        # Load & Merge
        trial_data = {}
        valid = True
        for m in MODELS:
            d = load_single_prediction(trial, m)
            if d is None: valid=False; break
            trial_data[m] = restore_and_merge(d, orig_data, pm, sm)
        if not valid: continue
        
        base = trial_data[MODELS[0]].copy()
        for m in MODELS:
            s = trial_data[m][["timestamp", "signal"]].rename(columns={"signal": f"sig_{m}"})
            base = pd.merge(base, s, on="timestamp", how="left").fillna(0)
            
        # Generate Combinations
        combos = []
        for r in range(1, len(MODELS)+1): combos.extend(combinations(MODELS, r))
        
        for combo in combos:
            c_name = "+".join(combo)
            col = f"ens_{c_name}"
            n = len(combo)
            thr = (n // 2) + 1
            base[col] = (base[[f"sig_{m}" for m in combo]].sum(axis=1) >= thr).astype(int)
            
            # Quick Dev Backtest (for Ranking)
            dev_sub = base[base["fold_type"] != "final_holdout"]
            if len(dev_sub) == 0: continue
            res = backtest_spot_pyramiding(dev_sub, FEE_RATE, MAX_PYRAMIDING, signal_col=col)
            
            all_competitors.append({
                "Name": f"{trial.name}_{c_name}",
                "Dev_Ret": res["Return"],
                "Data": base, "Signal_Col": col
            })

    # 2. Top 20 선정
    print("\n\n3. Selecting Top 20 & Generating Detail Reports...")
    top20 = sorted(all_competitors, key=lambda x: x["Dev_Ret"], reverse=True)[:20]
    
    rank_summary = []
    
    for i, p in enumerate(top20):
        rank = i + 1
        print(f"   [{rank}] Generating report for {p['Name']}...")
        
        df = p["Data"]
        sig = p["Signal_Col"]
        
        # Fold별 상세 분석
        fold_details = []
        # Dev Folds
        for fold, grp in df[df["fold_type"] != "final_holdout"].groupby("fold"):
            strat = backtest_spot_pyramiding(grp, FEE_RATE, MAX_PYRAMIDING, signal_col=sig)
            bh = calculate_bh_metrics_detailed(grp, FEE_RATE)
            
            fold_details.append({
                "Period": fold,
                "Model Return": strat["Return"], "BH Return": bh["Return"],
                "Model MDD": strat["MDD"], "BH MDD": bh["MDD"],
                "Win Rate": strat["WinRate"], "Sharpe": strat["Sharpe"], "Trades": strat["Trades"]
            })
            
        # Final Holdout
        hold = df[df["fold_type"] == "final_holdout"]
        if len(hold) > 0:
            h_strat = backtest_spot_pyramiding(hold, FEE_RATE, MAX_PYRAMIDING, signal_col=sig)
            h_bh = calculate_bh_metrics_detailed(hold, FEE_RATE)
            
            fold_details.append({
                "Period": "★ FINAL_HOLDOUT",
                "Model Return": h_strat["Return"], "BH Return": h_bh["Return"],
                "Model MDD": h_strat["MDD"], "BH MDD": h_bh["MDD"],
                "Win Rate": h_strat["WinRate"], "Sharpe": h_strat["Sharpe"], "Trades": h_strat["Trades"]
            })
            
            # Summary용 데이터
            rank_summary.append({
                "Rank": rank, "Model": p["Name"],
                "Dev Return": p["Dev_Ret"],
                "Holdout Return": h_strat["Return"],
                "Holdout MDD": h_strat["MDD"],
                "Report File": f"details_{rank}_{p['Name']}.csv"
            })
            
        # Save Detailed CSV
        det_df = pd.DataFrame(fold_details)
        # Format
        for c in ["Model Return", "BH Return", "Model MDD", "BH MDD", "Win Rate"]:
            det_df[c] = det_df[c].apply(lambda x: f"{x*100:.2f}%")
        det_df["Sharpe"] = det_df["Sharpe"].apply(lambda x: f"{x:.4f}")
        
        # Clean Filename
        safe_name = p["Name"].replace("|", "_").replace("+", "_").replace(" ", "")
        det_df.to_csv(REPORT_DIR / f"rank_{rank}_{safe_name}.csv", index=False)

    # 3. Summary 저장
    summ_df = pd.DataFrame(rank_summary)
    # Format
    for c in ["Dev Return", "Holdout Return", "Holdout MDD"]:
        summ_df[c] = summ_df[c].apply(lambda x: f"{x*100:.2f}%")
        
    print("\n" + "="*100)
    print("🏆 TOP 20 RANKING (Reports Generated)")
    print("="*100)
    print(summ_df[["Rank", "Model", "Dev Return", "Holdout Return", "Holdout MDD"]].to_string(index=False))
    
    summ_df.to_csv(BASE_DIR / "sniper_rank_summary.csv", index=False)
    print(f"\nCheck '{REPORT_DIR}' folder for detailed reports!")


1. Loading Original Data...

2. Creating Competitors...
   Processing T9_L9_P1.8_S0.5_N25.....

3. Selecting Top 20 & Generating Detail Reports...
   [1] Generating report for T7_L3_P1.6_S0.7_N10_XGBoost...


KeyError: 'WinRate'